<br>
<font size=7 color='blue' face='Candara'>
    <center>
        <b>Tracking CoronaVirus Around the World</b>
    </center>
</font>
<br>

In [184]:
%reset -f

In [185]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import numpy as np
import pandas as pd

from collections import Counter

import re

from ipywidgets import interact, interactive, widgets
from IPython.display import display

from colorama import Style, Fore, Back

In [186]:
daysOfWeek = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
monthsOfYear = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def getFullDate(dtm):
    wkdy = daysOfWeek[dtm.dayofweek][:3]
    mth = monthsOfYear[dtm.month - 1]
    dy = str(dtm.day)
    if len(dy)==1:
        dy = '0'+dy
    yr = str(dtm.year)
    return (wkdy+', '+mth+' '+dy+' '+yr)

## Load Datasets

In [187]:
urlGlobalConf = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
urlGlobalDeaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
urlGlobalRecovery = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'

dfConfGlobal = pd.read_csv(urlGlobalConf)
dfDeadGlobal = pd.read_csv(urlGlobalDeaths)
dfRecGlobal = pd.read_csv(urlGlobalRecovery)

# dfConfGlobal = pd.read_csv('time_series_covid19_confirmed_global.csv')
# dfDeadGlobal = pd.read_csv('time_series_covid19_deaths_global.csv')
# dfRecGlobal = pd.read_csv('time_series_covid19_recovered_global.csv')

theDatesGlobal = re.findall(pattern='[0-9]{1,2}/[0-9]{1,2}/[0-9]{1,2}', string=str(dfConfGlobal.columns))

# =======================================================================================================================

colsToDrop = ['Lat','Long_', 'iso2', 'iso3', 'code3', 'UID', 'FIPS']

urlConfUS = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
urlDeathsUS = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

dfConfUS = pd.read_csv(urlConfUS)
dfDeadUS = pd.read_csv(urlDeathsUS)

# dfConfUS = pd.read_csv('time_series_covid19_confirmed_US.csv')
# dfConfUS = dfConfUS.drop(colsToDrop, axis=1)

# dfDeadUS = pd.read_csv('time_series_covid19_deaths_US.csv')
# dfDeadUS = dfDeadUS.drop(colsToDrop, axis=1)

theDatesUS = re.findall(pattern='[0-9]{1,2}/[0-9]{1,2}/[0-9]{1,2}', string=str(dfConfUS.columns))

## Summary from Latest Global Data

In [188]:
def getCountrySummary_New(Country):
    
    idx = [True if i==Country else False for i in dfConfGlobal['Country/Region']]
    cdf = dfConfGlobal[idx]

    idx = [True if i==Country else False for i in dfDeadGlobal['Country/Region']]
    ddf = dfDeadGlobal[idx]

    idx = [True if i==Country else False for i in dfRecGlobal['Country/Region']]
    rdf = dfRecGlobal[idx]

    cc = cdf[theDatesGlobal[-1]].sum()
    dd = ddf[theDatesGlobal[-1]].sum()
    rr = rdf[theDatesGlobal[-1]].sum()
    
    print('Figures at a Glance (' + Country + ')\n\tConfirmed:\t', cc, '\n\tDeaths:\t\t', dd, '\n\tRecovered:\t', rr)

# ====================================================================================

summaryCountryList = sorted(list(set(dfConfGlobal['Country/Region'])))

interact(getCountrySummary_New, Country=summaryCountryList);

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

In [189]:
def getCountryTotals_New(Sort_By):
    res = []
    for cntry in set(dfConfGlobal['Country/Region']):
        
        idx = [True if i==cntry else False for i in dfConfGlobal['Country/Region']]
        cdf = dfConfGlobal[idx]
        
        idx = [True if i==cntry else False for i in dfDeadGlobal['Country/Region']]
        ddf = dfDeadGlobal[idx]
        
        idx = [True if i==cntry else False for i in dfRecGlobal['Country/Region']]
        rdf = dfRecGlobal[idx]
        
        cc = cdf[theDatesGlobal[-1]].sum()
        dd = ddf[theDatesGlobal[-1]].sum()
        rr = rdf[theDatesGlobal[-1]].sum()
        deathRate = round(dd/cc, 2)
        recoveryRate = round(rr/cc, 2)
        
        res.append( (cntry, cc, dd, rr, deathRate, recoveryRate) )
        
    sorter = {'Confirmed': 1, 'Deaths': 2, 'Recoveries': 3, 'Death_Rate': 4, 'Recovery_Rate': 5}
    res = sorted(res, key=lambda x:x[sorter[Sort_By]], reverse=True)
    return pd.DataFrame(res, columns=['Country', 'Cases', 'Deaths', 'Recoveries', 'Death_Rate', 'Recovery_Rate']).head(26)

# ==================================================================

sortList = ['Confirmed', 'Deaths', 'Recoveries', 'Death_Rate', 'Recovery_Rate']

interact(getCountryTotals_New, Sort_By=sortList);

interactive(children=(Dropdown(description='Sort_By', options=('Confirmed', 'Deaths', 'Recoveries', 'Death_Rat…

## Global COVID Data

In [190]:
# function to plot the country-level outcomes (cases/deaths/recoveries)

def plotGlobalOutcome(thisCountry, thisOutcome='cases'):
    
    if thisOutcome=='death':
        idx = [True if (i==thisCountry) else False for i in dfDeadGlobal['Country/Region']]
        df = dfDeadGlobal[idx]
        theOutcome = 'Deaths'
        theColor = 'red'
    elif thisOutcome=='recovery':
        idx = [True if (i==thisCountry) else False for i in dfRecGlobal['Country/Region']]
        df = dfRecGlobal[idx]
        theOutcome = 'Recoveries'
        theColor = 'C0'
    else:
        idx = [True if (i==thisCountry) else False for i in dfConfGlobal['Country/Region']]
        df = dfConfGlobal[idx]
        theOutcome = 'Cases'
        theColor = 'C1'

    res = []

    for i in theDatesGlobal:
        ss = df[i].sum()
        res.append( (i, ss) )

    theCountry = list(set(df['Country/Region']))[0]

    px = [i[0] for i in res]
    py = [i[1] for i in res]

    plt.figure(figsize=[22,10])
    plt.bar(px,py, color=theColor)
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# '+theOutcome, fontsize=30)
    plt.title('Confirmed ' + theOutcome + ' in ' + theCountry, fontsize=35)
    plt.show()

In [191]:
# function to plot the country-level curves (active cases)

def plotCurveGlobal(thisCountry):
    
    idx = [True if i==thisCountry else False for i in dfConfGlobal['Country/Region']]
    cdf = dfConfGlobal[idx]
    
    idx = [True if i==thisCountry else False for i in dfDeadGlobal['Country/Region']]
    ddf = dfDeadGlobal[idx]
    
    idx = [True if i==thisCountry else False for i in dfRecGlobal['Country/Region']]
    rdf = dfRecGlobal[idx]

    res = []

    for dt in theDatesGlobal:
        ss = cdf[dt].sum() - ddf[dt].sum() - rdf[dt].sum()
        res.append( (dt, ss) )

    theCountry = list(set(cdf['Country/Region']))[0]

    px = [i[0] for i in res]
    py = [i[1] for i in res]

    plt.figure(figsize=[22,10])
    plt.bar(px,py, color='C4')
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# Active Cases', fontsize=30)
    plt.title('Active Cases in ' + theCountry, fontsize=35)
    plt.show()

In [192]:
def showCountry(Country):
    plotGlobalOutcome(Country)
    plotGlobalOutcome(Country, 'death')
    plotGlobalOutcome(Country, 'recovery')

theCountryList = sorted(list(set(dfRecGlobal['Country/Region'])))

interact(showCountry, Country=theCountryList);

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

In [193]:
def showCurve(Country):
    plotCurveGlobal(Country)

theCountryList = sorted(list(set(dfRecGlobal['Country/Region'])))

interact(showCurve, Country=theCountryList);

interactive(children=(Dropdown(description='Country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

## USA COVID Data

In [194]:
# function to plot the state-level outcomes (cases/deaths)

def plotOutcome(thisState, thisOutcome='cases'):
    
    if thisOutcome=='deaths':
        idx = [True if (i==thisState) else False for i in dfDeadUS['Province_State']]
        df = dfDeadUS[idx]
        theOutcome = 'Deaths'
        theColor = 'red'
    else:
        idx = [True if (i==thisState) else False for i in dfConfUS['Province_State']]
        df = dfConfUS[idx]
        theOutcome = 'Cases'
        theColor = 'C1'

    res = []

    for i in theDatesUS:
        ss = df[i].sum()
        res.append( (i, ss) )

    px = [i[0] for i in res]
    py = [i[1] for i in res]

    plt.figure(figsize=[22,10])
    plt.bar(px,py, color=theColor)
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# '+theOutcome, fontsize=30)
    plt.title('Confirmed ' + theOutcome + ' in ' + thisState, fontsize=35)
    plt.show()

In [195]:
theStateList = sorted(list(set(dfConfUS['Province_State'])))

def showState(State):
    plotOutcome(State, thisOutcome='cases')
    plotOutcome(State, thisOutcome='deaths')

interact(showState, State=theStateList);

interactive(children=(Dropdown(description='State', options=('Alabama', 'Alaska', 'American Samoa', 'Arizona',…

## Texas COVID Data

In [196]:
idx = [True if i=='Texas' else False for i in dfConfUS['Province_State']]
dfConfTexas = dfConfUS[idx]

idx = [True if i=='Texas' else False for i in dfDeadUS['Province_State']]
dfDeadTexas = dfDeadUS[idx]

theDatesTexas = re.findall(pattern='[0-9]{1,2}/[0-9]{1,2}/[0-9]{1,2}', string=str(dfConfTexas.columns))

In [197]:
# function to plot the county-level outcomes (cases/deaths) for Texas

def plotTexasCountyOutcome(County):
    
    idx = [True if (i==County) else False for i in dfConfTexas['Admin2']]
    cdf = dfConfTexas[idx]
    
    idx = [True if (i==County) else False for i in dfDeadTexas['Admin2']]
    ddf = dfDeadTexas[idx]

    res = []

    for i in theDatesTexas:
        cc = cdf[i].sum()
        dd = ddf[i].sum()
        res.append( (i, cc, dd) )

    px = [i[0] for i in res]
    py = [i[1] for i in res]
    pz = [i[2] for i in res]

    plt.figure(figsize=[22,10])
    plt.plot(px,py, color='C1', linewidth=5)
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# Cases', fontsize=30)
    plt.title('Confirmed Cases in ' + County.upper() + ' County', fontsize=35)
    plt.show()
    
    plt.figure(figsize=[22,10])
    plt.plot(px,pz, color='red', linewidth=5)
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# Deaths', fontsize=30)
    plt.title('Confirmed Deaths in ' + County.upper() + ' County', fontsize=35)
    plt.show()

In [198]:
theCountyList = sorted(list(set(dfConfTexas['Admin2'])))

interact(plotTexasCountyOutcome, County=theCountyList);

interactive(children=(Dropdown(description='County', options=('Anderson', 'Andrews', 'Angelina', 'Aransas', 'A…

## COVID Reports for States and Counties

In [199]:
xdf = dfConfUS.copy()
zdf = dfDeadUS.copy()

xdf['Admin2'] = ['Unknown_'+i[0] if i[1]!=i[1] else i[1] for i in zip(xdf['Province_State'], xdf['Admin2'])]
zdf['Admin2'] = ['Unknown_'+i[0] if i[1]!=i[1] else i[1] for i in zip(zdf['Province_State'], zdf['Admin2'])]

In [200]:
# function to plot the county-level outcomes (cases/deaths) for US States

def plotStateCountyOutcome(State, County):
    
    idx = [True if i==State else False for i in xdf['Province_State']]
    cdf = xdf[idx]
    
    idx = [True if i==State else False for i in zdf['Province_State']]
    ddf = zdf[idx]
    
    if County not in list(cdf['Admin2']):
        County = np.random.choice(cdf['Admin2'])
    
    idx = [True if (i==County) else False for i in cdf['Admin2']]
    cdf = cdf[idx]
    
    idx = [True if (i==County) else False for i in ddf['Admin2']]
    ddf = ddf[idx]

    res = []

    for i in theDatesUS:
        cc = cdf[i].sum()
        dd = ddf[i].sum()
        res.append( (i, cc, dd) )

    px = [i[0] for i in res]
    py = [i[1] for i in res]
    pz = [i[2] for i in res]

    plt.figure(figsize=[22,10])
    plt.plot(px,py, color='C1', linewidth=5)
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# Cases', fontsize=30)
    plt.title('Confirmed Cases in ' + County.upper() + ' County, '+State.upper(), fontsize=35)
    plt.show()
    
    plt.figure(figsize=[22,10])
    plt.plot(px,pz, color='red', linewidth=5)
    plt.xticks(ticks=range(len(px)), labels=[getFullDate(i) for i in pd.to_datetime(px)], fontsize=15, rotation=90)
    plt.yticks(fontsize=25, rotation=0)
    plt.ylabel('# Deaths', fontsize=30)
    plt.title('Confirmed Deaths in ' + County.upper() + ' County, '+State.upper(), fontsize=35)
    plt.show()

In [201]:
tt = zip(dfConfUS['Province_State'], dfConfUS['Admin2'])
tt = [(i,j) if j==j else (i, 'Unknown_'+i) for i,j in tt]

dictLoc = dict()

for i in tt:
    try:
        dictLoc[i[0]].append(i[1])
    except KeyError:
        dictLoc[i[0]] = [i[1]]

In [202]:
def populateCounties(State):
    countyOptions.options = dictLoc[State]

def plotCounty(County):
    plotStateCountyOutcome(i.kwargs['State'], County)

stateOptions = widgets.Dropdown(options=dictLoc.keys())
# defaultState = stateOptions.value

# countyOptions = widgets.Dropdown(options=dictLoc[defaultState])
countyOptions = widgets.Dropdown()

i = interactive(populateCounties, State=stateOptions)
j = interactive(plotCounty, County=countyOptions)

display(i)
display(j)

interactive(children=(Dropdown(description='State', options=('American Samoa', 'Guam', 'Northern Mariana Islan…

interactive(children=(Dropdown(description='County', options=('Unknown_American Samoa',), value='Unknown_Ameri…